BRONZE LAYER

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Bronze layer").getOrCreate()

In [ ]:
base_url = "https://raw.githubusercontent.com/ARsenic100/sigmoid/main/"
files = ["CUST_AZ12.csv", "LOC_A101.csv", "PX_CAT_G1V2.csv", "cust_info.csv", "prd_info.csv", "sales_details.csv"]

In [ ]:
import requests

for file in files:
    r = requests.get(base_url + file)
    with open(file, "wb") as f:
        f.write(r.content)

print("All files downloaded successfully.")

All files downloaded successfully.


In [ ]:
cust_az12 = spark.read.option("header", True).option("inferSchema", True).csv("CUST_AZ12.csv")
cust_az12.show(5)

+-------------+----------+------+
|          CID|     BDATE|   GEN|
+-------------+----------+------+
|NASAW00011000|1971-10-06|  Male|
|NASAW00011001|1976-05-10|  Male|
|NASAW00011002|1971-02-09|  Male|
|NASAW00011003|1973-08-14|Female|
|NASAW00011004|1979-08-05|Female|
+-------------+----------+------+
only showing top 5 rows


In [ ]:
LOC_A101 = spark.read.option("header", True).option("inferSchema", True).csv("LOC_A101.csv")
LOC_A101.show(5)

+-----------+---------+
|        CID|    CNTRY|
+-----------+---------+
|AW-00011000|Australia|
|AW-00011001|Australia|
|AW-00011002|Australia|
|AW-00011003|Australia|
|AW-00011004|Australia|
+-----------+---------+
only showing top 5 rows


In [ ]:
PX_CAT_G1V2 = spark.read.option("header", True).option("inferSchema", True).csv("PX_CAT_G1V2.csv")
PX_CAT_G1V2.show(5)

+-----+-----------+-----------------+-----------+
|   ID|        CAT|           SUBCAT|MAINTENANCE|
+-----+-----------+-----------------+-----------+
|AC_BR|Accessories|       Bike Racks|        Yes|
|AC_BS|Accessories|      Bike Stands|         No|
|AC_BC|Accessories|Bottles and Cages|         No|
|AC_CL|Accessories|         Cleaners|        Yes|
|AC_FE|Accessories|          Fenders|         No|
+-----+-----------+-----------------+-----------+
only showing top 5 rows


In [ ]:
cust_info = spark.read.option("header", True).option("inferSchema", True).csv("cust_info.csv")
cust_info.show(5)

+------+----------+-------------+------------+------------------+--------+---------------+
|cst_id|   cst_key|cst_firstname|cst_lastname|cst_marital_status|cst_gndr|cst_create_date|
+------+----------+-------------+------------+------------------+--------+---------------+
| 11000|AW00011000|          Jon|       Yang |                 M|       M|     2025-10-06|
| 11001|AW00011001|       Eugene|     Huang  |                 S|       M|     2025-10-06|
| 11002|AW00011002|        Ruben|      Torres|                 M|       M|     2025-10-06|
| 11003|AW00011003|      Christy|         Zhu|                 S|       F|     2025-10-06|
| 11004|AW00011004|    Elizabeth|     Johnson|                 S|       F|     2025-10-06|
+------+----------+-------------+------------+------------------+--------+---------------+
only showing top 5 rows


In [ ]:
prd_info = spark.read.option("header", True).option("inferSchema", True).csv("prd_info.csv")
prd_info.show(5)

+------+----------------+--------------------+--------+--------+------------+----------+
|prd_id|         prd_key|              prd_nm|prd_cost|prd_line|prd_start_dt|prd_end_dt|
+------+----------------+--------------------+--------+--------+------------+----------+
|   210|CO-RF-FR-R92B-58|HL Road Frame - B...|    NULL|      R |  2003-07-01|      NULL|
|   211|CO-RF-FR-R92R-58|HL Road Frame - R...|    NULL|      R |  2003-07-01|      NULL|
|   212| AC-HE-HL-U509-R|Sport-100 Helmet-...|      12|      S |  2011-07-01|2007-12-28|
|   213| AC-HE-HL-U509-R|Sport-100 Helmet-...|      14|      S |  2012-07-01|2008-12-27|
|   214| AC-HE-HL-U509-R|Sport-100 Helmet-...|      13|      S |  2013-07-01|      NULL|
+------+----------------+--------------------+--------+--------+------------+----------+
only showing top 5 rows


In [ ]:
sales_details = spark.read.option("header", True).option("inferSchema", True).csv("sales_details.csv")
sales_details.show(5)

+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|sls_ord_num|sls_prd_key|sls_cust_id|sls_order_dt|sls_ship_dt|sls_due_dt|sls_sales|sls_quantity|sls_price|
+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|    SO43697| BK-R93R-62|      21768|    20101229|   20110105|  20110110|     3578|           1|     3578|
|    SO43698| BK-M82S-44|      28389|    20101229|   20110105|  20110110|     3400|           1|     3400|
|    SO43699| BK-M82S-44|      25863|    20101229|   20110105|  20110110|     3400|           1|     3400|
|    SO43700| BK-R50B-62|      14501|    20101229|   20110105|  20110110|      699|           1|      699|
|    SO43701| BK-M82S-44|      11003|    20101229|   20110105|  20110110|     3400|           1|     3400|
+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
only showing top 5 rows


In [ ]:
bronze_path="bronze/"

cust_az12.write.mode("overwrite").parquet(bronze_path+'bronze_CUST_AZ12')
LOC_A101.write.mode("overwrite").parquet(bronze_path+'bronze_LOC_A101')
PX_CAT_G1V2.write.mode("overwrite").parquet(bronze_path+'bronze_PX_CAT_G1V2')
cust_info.write.mode("overwrite").parquet(bronze_path+'bronze_cust_info')
prd_info.write.mode("overwrite").parquet(bronze_path+'bronze_prd_info')
sales_details.write.mode("overwrite").parquet(bronze_path+'bronze_sales_details')


SILVER LAYER


In [ ]:
crm_cust_info = spark.read.parquet(bronze_path + "bronze_cust_info")
crm_prd_info = spark.read.parquet(bronze_path + "bronze_prd_info")
crm_sales_details = spark.read.parquet(bronze_path + "bronze_sales_details")
erp_cust_az12 = spark.read.parquet(bronze_path + "bronze_CUST_AZ12")
erp_loc_a101 = spark.read.parquet(bronze_path + "bronze_LOC_A101")
erp_px_cat_g1v2 = spark.read.parquet(bronze_path + "bronze_PX_CAT_G1V2")


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

window_spec = Window.partitionBy("cst_id").orderBy(col("cst_create_date").desc())

silver_crm_cust_info = (
    crm_cust_info
    .filter(col("cst_id").isNotNull())
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
    .withColumn("cst_firstname", trim(col("cst_firstname")))
    .withColumn("cst_lastname", trim(col("cst_lastname")))
    .withColumn(
        "cst_marital_status",
        when(upper(trim(col("cst_marital_status"))) == "S", "Single")
        .when(upper(trim(col("cst_marital_status"))) == "M", "Married")
        .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        when(upper(trim(col("cst_gndr"))) == "F", "Female")
        .when(upper(trim(col("cst_gndr"))) == "M", "Male")
        .otherwise("n/a")
    )
    .withColumn("dwh_create_date", current_timestamp())
)

In [ ]:
window_prd = Window.partitionBy(expr("substring(prd_key,7)")).orderBy("prd_start_dt")

silver_crm_prd_info = (
    crm_prd_info
    .withColumn("cat_id", regexp_replace(substring("prd_key", 1, 5), "-", "_"))
    .withColumn("prd_key", substring("prd_key", 7, 100))
    .withColumn("prd_cost", coalesce(col("prd_cost"), lit(0)))
    .withColumn(
        "prd_line",
        when(upper(trim(col("prd_line"))) == "M", "Mountain")
        .when(upper(trim(col("prd_line"))) == "R", "Road")
        .when(upper(trim(col("prd_line"))) == "S", "Other Sales")
        .when(upper(trim(col("prd_line"))) == "T", "Touring")
        .otherwise("n/a")
    )
    .withColumn("prd_start_dt", col("prd_start_dt").cast("date"))
    .withColumn(
        "prd_end_dt",
        date_sub(lead("prd_start_dt").over(window_prd), 1)
    )
    .withColumn("dwh_create_date", current_timestamp())
)


In [ ]:
silver_crm_sales_details = (
    crm_sales_details
    .withColumn(
        "sls_order_dt",
        when((col("sls_order_dt") == 0) | (length(col("sls_order_dt").cast("string")) != 8), None)
        .otherwise(to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        when((col("sls_ship_dt") == 0) | (length(col("sls_ship_dt").cast("string")) != 8), None)
        .otherwise(to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        when((col("sls_due_dt") == 0) | (length(col("sls_due_dt").cast("string")) != 8), None)
        .otherwise(to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_sales",
        when(
            (col("sls_sales").isNull()) |
            (col("sls_sales") <= 0) |
            (col("sls_sales") != col("sls_quantity") * abs(col("sls_price"))),
            col("sls_quantity") * abs(col("sls_price"))
        ).otherwise(col("sls_sales"))
    )
    .withColumn(
        "sls_price",
        when(
            (col("sls_price").isNull()) | (col("sls_price") <= 0),
            col("sls_sales") / col("sls_quantity")
        ).otherwise(col("sls_price"))
    )
    .withColumn("dwh_create_date", current_timestamp())
)


In [ ]:
silver_erp_cust_az12 = (
    erp_cust_az12
    .withColumn(
        "cid",
        when(col("cid").startswith("NAS"), substring("cid", 4, 100))
        .otherwise(col("cid"))
    )
    .withColumn(
        "bdate",
        when(col("bdate") > current_date(), None)
        .otherwise(col("bdate"))
    )
    .withColumn(
        "gen",
        when(upper(trim(col("gen"))).isin("F", "FEMALE"), "Female")
        .when(upper(trim(col("gen"))).isin("M", "MALE"), "Male")
        .otherwise("n/a")
    )
    .withColumn("dwh_create_date", current_timestamp())
)

In [ ]:
silver_erp_loc_a101 = (
    erp_loc_a101
    .withColumn("cid", regexp_replace("cid", "-", ""))
    .withColumn(
        "cntry",
        when(trim(col("cntry")) == "DE", "Germany")
        .when(trim(col("cntry")).isin("US", "USA"), "United States")
        .when((col("cntry").isNull()) | (trim(col("cntry")) == ""), "n/a")
        .otherwise(trim(col("cntry")))
    )
    .withColumn("dwh_create_date", current_timestamp())
)

In [ ]:
silver_erp_px_cat_g1v2 = (
    erp_px_cat_g1v2
    .withColumn("dwh_create_date", current_timestamp())
)

In [ ]:
silver_path = "silver/"
silver_erp_cust_az12.write.mode("overwrite").parquet(silver_path + 'silver_layer_CUST_AZ12')
silver_erp_loc_a101.write.mode("overwrite").parquet(silver_path + 'silver_layer_LOC_A101')
silver_erp_px_cat_g1v2.write.mode("overwrite").parquet(silver_path + 'silver_layer_PX_CAT_G1V2')
silver_crm_cust_info.write.mode("overwrite").parquet(silver_path + 'silver_layer_cust_info')
silver_crm_prd_info.write.mode("overwrite").parquet(silver_path + 'silver_layer_prd_info')
silver_crm_sales_details.write.mode("overwrite").parquet(silver_path + 'silver_layer_sales_details')

GOLD LAYER

In [ ]:
crm_cust_info = spark.read.parquet(silver_path + "silver_layer_cust_info")
crm_prd_info = spark.read.parquet(silver_path + "silver_layer_prd_info")
crm_sales_details = spark.read.parquet(silver_path + "silver_layer_sales_details")
erp_cust_az12 = spark.read.parquet(silver_path + "silver_layer_CUST_AZ12")
erp_loc_a101 = spark.read.parquet(silver_path + "silver_layer_LOC_A101")
erp_px_cat_g1v2 = spark.read.parquet(silver_path + "silver_layer_PX_CAT_G1V2")


In [ ]:
dim_customers = (
    crm_cust_info.alias("ci")
    .join(erp_cust_az12.alias("ca"),
          col("ci.cst_key") == col("ca.cid"),
          "left")
    .join(erp_loc_a101.alias("la"),
          col("ci.cst_key") == col("la.cid"),
          "left")
    .select(
        col("ci.cst_id").alias("customer_id"),
        col("ci.cst_key").alias("customer_number"),
        col("ci.cst_firstname").alias("first_name"),
        col("ci.cst_lastname").alias("last_name"),
        col("la.cntry").alias("country"),
        col("ci.cst_marital_status").alias("marital_status"),
        when(col("ci.cst_gndr") != "n/a", col("ci.cst_gndr"))
            .otherwise(coalesce(col("ca.gen"), lit("n/a")))
            .alias("gender"),
        col("ca.bdate").alias("birthdate"),
        col("ci.cst_create_date").alias("create_date")
    )
)

win_func = Window.orderBy("customer_id")

dim_customers = dim_customers.withColumn("customer_key", row_number().over(win_func))
dim_customers = dim_customers.select(
    "customer_key",
    "customer_id",
    "customer_number",
    "first_name",
    "last_name",
    "country",
    "marital_status",
    "gender",
    "birthdate",
    "create_date"
)

In [ ]:
dim_products = (
    crm_prd_info.alias("pn")
    .join(erp_px_cat_g1v2.alias("pc"),
          col("pn.cat_id") == col("pc.id"),
          "left")
    .filter(col("pn.prd_end_dt").isNull())
    .select(
        col("pn.prd_id").alias("product_id"),
        col("pn.prd_key").alias("product_number"),
        col("pn.prd_nm").alias("product_name"),
        col("pn.cat_id").alias("category_id"),
        col("pc.cat").alias("category"),
        col("pc.subcat").alias("subcategory"),
        col("pc.maintenance").alias("maintenance"),
        col("pn.prd_cost").alias("cost"),
        col("pn.prd_line").alias("product_line"),
        col("pn.prd_start_dt").alias("start_date")
    )
)

win_func = Window.orderBy("start_date", "product_number")

dim_products = dim_products.withColumn("product_key", row_number().over(win_func))
dim_products = dim_products.select(
    "product_key",
    "product_id",
    "product_number",
    "product_name",
    "category_id",
    "category",
    "subcategory",
    "maintenance",
    "cost",
    "product_line",
    "start_date"
)


In [ ]:
crm_sales_details = crm_sales_details.withColumn("sls_cust_id", col("sls_cust_id").cast("int"))
dim_customers = dim_customers.withColumn("customer_id", col("customer_id").cast("int"))

fact_sales = (
    crm_sales_details.alias("sd")
    .join(dim_products.alias("pr"),
          trim(upper(col("sd.sls_prd_key"))) == trim(upper(col("pr.product_number"))),
          "left")
    .join(dim_customers.alias("cu"),
          col("sd.sls_cust_id") == col("cu.customer_id"),
          "inner")
    .select(
        col("sd.sls_ord_num").alias("order_number"),
        col("pr.product_key"),
        col("cu.customer_key"),
        col("sd.sls_order_dt").alias("order_date"),
        col("sd.sls_ship_dt").alias("shipping_date"),
        col("sd.sls_due_dt").alias("due_date"),
        col("sd.sls_sales").alias("sales_amount"),
        col("sd.sls_quantity").alias("quantity"),
        col("sd.sls_price").alias("price")
    )
)

In [ ]:
dim_customers.show()


+------------+-----------+---------------+----------+---------+-------------+--------------+------+----------+-----------+
|customer_key|customer_id|customer_number|first_name|last_name|      country|marital_status|gender| birthdate|create_date|
+------------+-----------+---------------+----------+---------+-------------+--------------+------+----------+-----------+
|           1|      11000|     AW00011000|       Jon|     Yang|    Australia|       Married|  Male|1971-10-06| 2025-10-06|
|           2|      11001|     AW00011001|    Eugene|    Huang|    Australia|        Single|  Male|1976-05-10| 2025-10-06|
|           3|      11002|     AW00011002|     Ruben|   Torres|    Australia|       Married|  Male|1971-02-09| 2025-10-06|
|           4|      11003|     AW00011003|   Christy|      Zhu|    Australia|        Single|Female|1973-08-14| 2025-10-06|
|           5|      11004|     AW00011004| Elizabeth|  Johnson|    Australia|        Single|Female|1979-08-05| 2025-10-06|
|           6|  

In [ ]:
dim_products.show()


+-----------+----------+--------------+--------------------+-----------+----------+---------------+-----------+----+------------+----------+
|product_key|product_id|product_number|        product_name|category_id|  category|    subcategory|maintenance|cost|product_line|start_date|
+-----------+----------+--------------+--------------------+-----------+----------+---------------+-----------+----+------------+----------+
|          1|       323|    BK-R50R-60|    Road-650 Red- 60|      BI_RB|     Bikes|     Road Bikes|        Yes| 487|        Road|2012-07-01|
|          2|       461|     SB-M891-L| Men's Bib-Shorts- L|      CL_BS|  Clothing|     Bib-Shorts|         No|  37| Other Sales|2012-07-01|
|          3|       460|     SB-M891-M| Men's Bib-Shorts- M|      CL_BS|  Clothing|     Bib-Shorts|         No|  37| Other Sales|2012-07-01|
|          4|       459|     SB-M891-S| Men's Bib-Shorts- S|      CL_BS|  Clothing|     Bib-Shorts|         No|  37| Other Sales|2012-07-01|
|          5|

In [ ]:
fact_sales.show()


+------------+-----------+------------+----------+-------------+----------+------------+--------+------+
|order_number|product_key|customer_key|order_date|shipping_date|  due_date|sales_amount|quantity| price|
+------------+-----------+------------+----------+-------------+----------+------------+--------+------+
|     SO43697|       NULL|       10769|2010-12-29|   2011-01-05|2011-01-10|        3578|       1|3578.0|
|     SO43698|       NULL|       17390|2010-12-29|   2011-01-05|2011-01-10|        3400|       1|3400.0|
|     SO43699|       NULL|       14864|2010-12-29|   2011-01-05|2011-01-10|        3400|       1|3400.0|
|     SO43700|       NULL|        3502|2010-12-29|   2011-01-05|2011-01-10|         699|       1| 699.0|
|     SO43701|       NULL|           4|2010-12-29|   2011-01-05|2011-01-10|        3400|       1|3400.0|
|     SO43702|       NULL|       16646|2010-12-30|   2011-01-06|2011-01-11|        3578|       1|3578.0|
|     SO43703|       NULL|        5625|2010-12-30|   20

1

In [ ]:
from pyspark.sql.functions import min, max, months_between, round

timeline = fact_sales.select(
    min("order_date").alias("first_order_date"),
    max("order_date").alias("last_order_date")
)

timeline = timeline.withColumn(
    "coverage_months",
    round(months_between("last_order_date", "first_order_date"), 1)
)

timeline.show()


+----------------+---------------+---------------+
|first_order_date|last_order_date|coverage_months|
+----------------+---------------+---------------+
|      2010-12-29|     2014-01-28|           37.0|
+----------------+---------------+---------------+



2

In [ ]:
from pyspark.sql.functions import current_date, floor

customer_age = dim_customers.withColumn(
    "age",
    floor(months_between(current_date(), "birthdate") / 12)
)

customer_age.select(
    min("age").alias("youngest_age"),
    max("age").alias("oldest_age")
).show()

+------------+----------+
|youngest_age|oldest_age|
+------------+----------+
|          39|       110|
+------------+----------+



3

In [ ]:
from pyspark.sql.functions import sum, avg

fact_sales.select(
    sum("sales_amount").alias("total_sales"),
    sum("quantity").alias("total_quantity"),
    avg("price").alias("avg_price")
).show()


+-----------+--------------+-----------------+
|total_sales|total_quantity|        avg_price|
+-----------+--------------+-----------------+
|   29356250|         60423|486.0377827080367|
+-----------+--------------+-----------------+



4

In [ ]:
dim_customers.groupBy("country").count().orderBy("count", ascending=False).show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 7482|
|     Australia| 3591|
|United Kingdom| 1913|
|        France| 1810|
|       Germany| 1780|
|        Canada| 1571|
|           n/a|  337|
+--------------+-----+



In [ ]:
dim_customers.groupBy("gender").count().orderBy("count", ascending=False).show()

+------+-----+
|gender|count|
+------+-----+
|  Male| 9341|
|Female| 9128|
|   n/a|   15|
+------+-----+



5

In [ ]:
dim_products.groupBy("category") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

+-----------+-----+
|   category|count|
+-----------+-----+
|      Bikes|   34|
|   Clothing|   20|
| Components|   17|
|Accessories|    3|
|       NULL|    1|
+-----------+-----+



In [ ]:
from pyspark.sql.functions import avg

dim_products.groupBy("category") \
    .agg(avg("cost").alias("avg_cost")) \
    .orderBy("avg_cost", ascending=False) \
    .show()

+-----------+------------------+
|   category|          avg_cost|
+-----------+------------------+
|      Bikes| 672.5882352941177|
| Components| 367.4117647058824|
|       NULL|              28.0|
|   Clothing|              26.4|
|Accessories|15.666666666666666|
+-----------+------------------+



6

In [ ]:
revenue_by_category = (
    fact_sales.join(dim_products, "product_key")
    .groupBy("category")
    .agg(sum("sales_amount").alias("revenue"))
    .orderBy("revenue", ascending=False)
)

revenue_by_category.show()


+-----------+-------+
|   category|revenue|
+-----------+-------+
|      Bikes|7806186|
|   Clothing| 217646|
|Accessories| 192775|
+-----------+-------+



In [ ]:
revenue_by_customer = (
    fact_sales.join(dim_customers, "customer_key")
    .groupBy("customer_id", "first_name", "last_name")
    .agg(sum("sales_amount").alias("revenue"))
    .orderBy("revenue", ascending=False)
)

revenue_by_customer.show()

+-----------+----------+---------+-------+
|customer_id|first_name|last_name|revenue|
+-----------+----------+---------+-------+
|      12132|   Kaitlyn|Henderson|  13294|
|      12301|   Nichole|     Nara|  13294|
|      12308|  Margaret|       He|  13268|
|      12131|   Randall|Dominguez|  13265|
|      12300|   Adriana| Gonzalez|  13242|
|      12321|      Rosa|       Hu|  13215|
|      12124|    Brandi|     Gill|  13195|
|      12307|      Brad|      She|  13172|
|      12296| Francisco|     Sara|  13164|
|      11433|   Maurice|     Shan|  12914|
|      11439|     Janet|    Munoz|  12488|
|      11241|      Lisa|      Cai|  11468|
|      11417|     Lacey|    Zheng|  11248|
|      11420|    Jordan|   Turner|  11200|
|      11242|     Larry|    Munoz|  11067|
|      12655|     Larry|  Vazquez|  10899|
|      13263|      Kate|    Anand|  10871|
|      12323|  Lawrence|   Alonso|  10836|
|      12333|  Terrance|Rodriguez|  10829|
|      12650|     Aaron|   Wright|  10813|
+----------

7

In [ ]:
top5_products = (
    fact_sales.join(dim_products, "product_key")
    .groupBy("product_name")
    .agg(sum("sales_amount").alias("revenue"))
    .orderBy("revenue", ascending=False)
    .limit(5)
)

top5_products.show()

+--------------------+-------+
|        product_name|revenue|
+--------------------+-------+
|Mountain-200 Blac...|1373454|
|Mountain-200 Blac...|1294854|
|    Road-250 Red- 58| 702666|
|Road-350-W Yellow...| 418446|
|Touring-1000 Yell...| 410048|
+--------------------+-------+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

product_revenue = (
    fact_sales.join(dim_products, "product_key")
    .groupBy("product_name")
    .agg(sum("sales_amount").alias("revenue"))
)

window_spec = Window.orderBy(product_revenue["revenue"].desc())

ranked_products = product_revenue.withColumn(
    "rank",
    rank().over(window_spec)
)

ranked_products.filter("rank <= 5").show()


+--------------------+-------+----+
|        product_name|revenue|rank|
+--------------------+-------+----+
|Mountain-200 Blac...|1373454|   1|
|Mountain-200 Blac...|1294854|   2|
|    Road-250 Red- 58| 702666|   3|
|Road-350-W Yellow...| 418446|   4|
|Touring-1000 Yell...| 410048|   5|
+--------------------+-------+----+



In [ ]:
product_revenue.orderBy("revenue").limit(5).show()


+--------------------+-------+
|        product_name|revenue|
+--------------------+-------+
|     Racing Socks- L|   2430|
|     Racing Socks- M|   2682|
|Half-Finger Glove...|  10632|
|Half-Finger Glove...|  11712|
|Half-Finger Glove...|  11976|
+--------------------+-------+



In [ ]:
top10_customers = (
    fact_sales.join(dim_customers, "customer_key")
    .groupBy("customer_id", "first_name", "last_name")
    .agg(sum("sales_amount").alias("revenue"))
    .orderBy("revenue", ascending=False)
    .limit(10)
)

top10_customers.show()


+-----------+----------+---------+-------+
|customer_id|first_name|last_name|revenue|
+-----------+----------+---------+-------+
|      12132|   Kaitlyn|Henderson|  13294|
|      12301|   Nichole|     Nara|  13294|
|      12308|  Margaret|       He|  13268|
|      12131|   Randall|Dominguez|  13265|
|      12300|   Adriana| Gonzalez|  13242|
|      12321|      Rosa|       Hu|  13215|
|      12124|    Brandi|     Gill|  13195|
|      12307|      Brad|      She|  13172|
|      12296| Francisco|     Sara|  13164|
|      11433|   Maurice|     Shan|  12914|
+-----------+----------+---------+-------+



In [ ]:
from pyspark.sql.functions import count

fewest_orders = (
    fact_sales.groupBy("customer_key")
    .agg(count("order_number").alias("order_count"))
    .filter("order_count > 0")
    .orderBy("order_count")
    .limit(3)
)

fewest_orders.show()

+------------+-----------+
|customer_key|order_count|
+------------+-----------+
|       15727|          1|
|        6336|          1|
|       16503|          1|
+------------+-----------+



In [ ]:
fewest_orders = (
    fact_sales.join(dim_customers, "customer_key")
    .groupBy("customer_id", "first_name", "last_name")
    .agg(count("order_number").alias("order_count"))
    .orderBy("order_count")
    .limit(3)
)

fewest_orders.show()

+-----------+----------+---------+-----------+
|customer_id|first_name|last_name|order_count|
+-----------+----------+---------+-----------+
|      12238|    Monica|    Raman|          1|
|      23938|    Harold|    Madan|          1|
|      22301|   Abigail|Rodriguez|          1|
+-----------+----------+---------+-----------+

